In [1]:
import struct

class Packet():
    '''Abstraction to handle the whole Confundo packet (e.g., with payload, if present)'''

    def __init__(self, payload=b"", isDup=False, seqNum=0, ackNum=0, connId=0, isAck=False, isSyn=False, isFin=False):
        self.seqNum = seqNum
        self.ackNum = ackNum
        self.connId = connId
        self.isAck = isAck
        self.isSyn = isSyn
        self.isFin = isFin
        self.payload = payload
        self.isDup = isDup # only for printing flags
        self.connId = connId

    def decode(self, fullPacket):
        (self.seqNum, self.ackNum, self.connId, flags) = struct.unpack("!IIHH", fullPacket[0:12])
        self.isAck = flags & (1 << 2)
        self.isSyn = flags & (1 << 1)
        self.isFin = flags & (1)
        self.payload = fullPacket[12:]
        return self

    def encode(self):
        flags = 0
        if self.isAck:
            flags = flags | (1 << 2)
        if self.isSyn:
            flags = flags | (1 << 1)
        if self.isFin:
            flags = flags | (1)
        pay = struct.pack("!IIHH",
                           self.seqNum, self.ackNum,
                           self.connId, flags)
        return pay + self.payload 
    
    
    
    
def format_line(command, pkt, cwnd, ssthresh):
    s = f"{command} {pkt.seqNum} {pkt.ackNum} {pkt.connId} {int(cwnd)} {ssthresh}"
    if pkt.isAck: s = s + " ACK"
    if pkt.isSyn: s = s + " SYN"
    if pkt.isFin: s = s + " FIN"
    if pkt.isDup: s = s + " DUP"
    return s

In [ ]:
import socket

 

localIP     = "127.0.0.1"

localPort   = 20001

bufferSize  = 1024


# Create a datagram socket

UDPServerSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)

 

# Bind to address and ip

UDPServerSocket.bind((localIP, localPort))

 

print("UDP server up and listening")

bytesAddressPair = UDPServerSocket.recvfrom(bufferSize)

message = bytesAddressPair[0]

address = bytesAddressPair[1]
    
inPkt = Packet().decode(message)

    
print(format_line("RECV",  inPkt, 1, 1))
pk = Packet(seqNum=77, ackNum=78, connId=inPkt.connId, isAck=True)
UDPServerSocket.sendto(pk.encode(), address)

data = b''

while True:

    bytesAddressPair = UDPServerSocket.recvfrom(bufferSize)

    message = bytesAddressPair[0]

    address = bytesAddressPair[1]
    
    inPkt = Packet().decode(message)

    data += inPkt.payload
    
    print(format_line("RECV",  inPkt, 1, 1))
    pk = Packet(seqNum=78, ackNum=inPkt.seqNum + len(inPkt.payload), connId=inPkt.connId, isAck=True)
    UDPServerSocket.sendto(pk.encode(), address)
    if(inPkt.isFin):
        break
print(data.decode("utf-8") )


UDP server up and listening
